# Hyperparameter tuning with AI Platform 

In [ ]:
import os
import time

## Configure environment
### Set a GCS bucket

In [ ]:
PROJECT_ID = !(gcloud config get-value core/project)
ARTIFACT_STORE = 'gs://{}-artifact-store'.format(PROJECT_ID[0])

In [ ]:
TRAINING_DATA_PATH = '{}/datasets/training.csv'.format(ARTIFACT_STORE)
TESTING_DATA_PATH = '{}/datasets/testing.csv'.format(ARTIFACT_STORE)
REGION = "us-central1"
JOBDIR_BUCKET = '{}/jobs'.format(ARTIFACT_STORE)

## Create a training application package
### Create a training module

In [ ]:
TRAINING_APP_FOLDER = '../hypertune_app/trainer'

os.makedirs(TRAINING_APP_FOLDER, exist_ok=True)
!touch $TRAINING_APP_FOLDER/__init__.py

In [ ]:
%%writefile $TRAINING_APP_FOLDER/train.py

import logging
import os
import subprocess
import sys

import fire
import numpy as np
import pandas as pd

import hypertune

from sklearn.model_selection import cross_val_score
from sklearn.externals import joblib
from sklearn.decomposition import PCA
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler


def train(job_dir, data_path, n_components, alpha):
    
    
  # Load data from GCS
  df_train = pd.read_csv(data_path)

  y = df_train.octane
  X = df_train.drop('octane', axis=1)
    
  # Configure a training pipeline
  pipeline = Pipeline([
    ('scale', StandardScaler()),
    ('reduce_dim', PCA(n_components=n_components)),
    ('regress', Ridge(alpha=alpha))
  ])

  # Calculate the performance metric
  scores = cross_val_score(pipeline, X, y, cv=10, scoring='neg_mean_squared_error')
    
  # Log it with hypertune
  hpt = hypertune.HyperTune()
  hpt.report_hyperparameter_tuning_metric(
    hyperparameter_metric_tag='neg_mean_squared_error',
    metric_value=scores.mean()
    )

  # Fit the model on a full dataset
  pipeline.fit(X, y)

  # Save the model
  model_filename = 'model.joblib'
  joblib.dump(value=pipeline, filename=model_filename)
  gcs_model_path = "{}/{}".format(job_dir, model_filename)
  subprocess.check_call(['gsutil', 'cp', model_filename, gcs_model_path], stderr=sys.stdout)
  logging.info("Saved model in: {}".format(gcs_model_path)) 
    
if __name__ == "__main__":
  logging.basicConfig(level=logging.INFO)
  fire.Fire(train)

### Create hyperparameter configuration file

In [ ]:
%%writefile $TRAINING_APP_FOLDER/hptuning_config.yaml 

trainingInput:
  hyperparameters:
    goal: MAXIMIZE
    maxTrials: 12
    maxParallelTrials: 3
    hyperparameterMetricTag: neg_mean_squared_error
    enableTrialEarlyStopping: TRUE 
    params:
    - parameterName: n_components
      type: DISCRETE
      discreteValues: [
          2,
          3,
          4,
          5,
          6,
          7,
          8
          ]
    - parameterName: alpha
      type: DOUBLE
      minValue:  0.0001
      maxValue:  0.1
      scaleType: UNIT_LINEAR_SCALE
      

### Configure dependencies

In [ ]:
%%writefile $TRAINING_APP_FOLDER/../setup.py

from setuptools import find_packages
from setuptools import setup

REQUIRED_PACKAGES = ['fire', 'gcsfs', 'cloudml-hypertune']

setup(
    name='trainer',
    version='0.1',
    install_requires=REQUIRED_PACKAGES,
    packages=find_packages(),
    include_package_data=True,
    description='My training application package.'
)

## Submit a training job

In [ ]:
JOB_NAME = "JOB_{}".format(time.strftime("%Y%m%d_%H%M%S"))
SCALE_TIER = "BASIC"
MODULE_NAME = "trainer.train"
RUNTIME_VERSION = "2.1"
PYTHON_VERSION = "3.7"

In [ ]:
!gcloud ai-platform jobs submit training $JOB_NAME \
--region $REGION \
--job-dir $JOBDIR_BUCKET/$JOB_NAME \
--package-path $TRAINING_APP_FOLDER \
--module-name $MODULE_NAME \
--scale-tier $SCALE_TIER \
--python-version $PYTHON_VERSION \
--runtime-version $RUNTIME_VERSION \
--config $TRAINING_APP_FOLDER/hptuning_config.yaml \
-- \
--data_path $TRAINING_DATA_PATH 

In [ ]:
!gcloud ai-platform jobs describe $JOB_NAME